<a id="stepSTART"></a>
**Проект**: Мобильное приложение с элементами социальной сети, маркетплейса, объединяющее взаимодействие 3 сегментов аудитории

**Контекст**: 
Для повышения доходимости до мероприятия участников, забронировавших участие в мероприятии необходимо разработать логику отправки и периодичность 
включения точек контакта 

**Задача**: 

- Провести анализ того, какова наиболее часто встречающаяся "граница бронирования" на мероприятие в зависимости от формата мероприятия

[Ключевые выводы](#step6)

[Рекомендации](#step7)

**Разделы исследования** <a id='stepend'></a>

1-2. [Импорт необходимых библиотек и формирование основного датафрейма](#step1)</n>

3. [Формирование датафрейма для анализа](#step2)
   
4. [Осмотр данных](#step3)

5. [Анализ поведения со статусом "Гость"](#step4)

6. [Анализ поведения со статусом "Амбассадор"](#step5)

7. [Ключевые выводы](#step6)

8. [Рекомендации](#step6)

### `1 - 2. Импорт необходимых библиотек и формирование основного датафрейма` <a id="step1"></a>

In [14]:
# 1. Импортируем необходимые библиотеки
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine

In [27]:
# 2. Сформируем датафрейм со всеми необходимыми данными
engine = create_engine('mysql+mysqlconnector://come_on_dude')

# Ваш SQL-запрос
query = """SELECT bookings.user_id,
        booked_users.booked_status_user,
        bookings.event_id,
        bookings.booking_date,
        events.start_date,
        events.format,
        events.type,
        events.author_id,
        booked_authors.booked_status_author,
        DATEDIFF(events.start_date, bookings.booking_date) AS diff_start,
        pass_status
  FROM (SELECT userId AS user_id,
               activityId AS event_id,
               DATE(createdAt) AS booking_date,
               withSpacePass AS pass_status
          FROM `krugi-prod-social`.activity_participant
         WHERE DATE(createdAt) >= "2024-12-01"
           AND deletedAt IS NULL
           AND userId NOT IN (SELECT id 
                                FROM `krugi-prod-social`.users
                               WHERE phone IN ('70333377777',
                                                '12640826663333',
                                                '72828203554',
                                                '79160631144',
                                                '78005553535',
                                                '79619919840',
                                                '79833856638',
                                                '79831063666',
                                                '79635088274',
                                                '71111111111',
                                                '78005553535')
                                  OR externalId = "crm")) AS bookings
        LEFT JOIN
        (SELECT id AS event_id,
                DATE(startDate) AS start_date,
                DATE(endDate) AS end_date,
                DATE(createdAt) AS created_date,
                format,
                type,
                authorId AS author_id
          FROM `krugi-prod-social`.activity
         WHERE DATE(createdAt) >= "2024-12-01"
          AND authorId NOT IN (SELECT id 
                                 FROM `krugi-prod-social`.users
                                WHERE phone IN ('70333377777',
                                                '12640826663333',
                                                '72828203554',
                                                '79160631144',
                                                '78005553535',
                                                '79619919840',
                                                '79833856638',
                                                '79831063666',
                                                '79635088274',
                                                '71111111111',
                                                '78005553535')
                                   OR externalId = "crm"
                                   OR clubMemberStatus = "MEMBER")) AS events
        ON bookings.event_id = events.event_id
        LEFT JOIN
        (SELECT id AS booked_user,
                clubMemberStatus AS booked_status_user
           FROM `krugi-prod-social`.users) AS booked_users
        ON bookings.user_id = booked_users.booked_user
        LEFT JOIN
        (SELECT id AS booked_author,
                clubMemberStatus AS booked_status_author
        FROM `krugi-prod-social`.users) AS booked_authors
        ON bookings.user_id = booked_authors.booked_author
WHERE events.start_date IS NOT NULL
  AND (DATEDIFF(events.start_date, bookings.booking_date) > 0)
  AND (DATEDIFF(events.end_date, bookings.booking_date) > 0 )
  AND bookings.user_id != events.author_id"""

# Выполнение запроса и загрузка результата в DataFrame
main_tab = pd.read_sql(query, engine)
main_tab.head(3)

,user_id,booked_status_user,event_id,booking_date,start_date,format,type,author_id,booked_status_author,diff_start,pass_status
0,5641685,AMBASSADOR,2038,2024-12-04,2024-12-22,OFFLINE,PAID,9927742,AMBASSADOR,18,1
1,5641685,AMBASSADOR,1890,2024-12-04,2024-12-11,ONLINE,PAID,38703,AMBASSADOR,7,1
2,5641685,AMBASSADOR,2082,2024-12-04,2025-01-22,OFFLINE,PAID,9930934,AMBASSADOR,49,1


### `3. Формирование датафрейма для анализа` <a id="step2"></a>

In [16]:
# 3. Сформируем датафрейм только с необходимыми колонками
bookings = main_tab.copy()
bookings = bookings[['user_id', 'booked_status_user','event_id', 
                     'format', 'type', 'pass_status', 
                     'booked_status_author', 'diff_start']]
bookings.head(3)

,user_id,booked_status_user,event_id,format,type,pass_status,booked_status_author,diff_start
0,5641685,AMBASSADOR,2038,OFFLINE,PAID,1,AMBASSADOR,18
1,5641685,AMBASSADOR,1890,ONLINE,PAID,1,AMBASSADOR,7
2,5641685,AMBASSADOR,2082,OFFLINE,PAID,1,AMBASSADOR,49


### `4. Осмотр данных` <a id="step3"></a>

In [17]:
# 4.1 Посмотрим сколько строк и колонок, адекватное ли кол-во исходя их контекста запроса
bookings.shape

(3051, 8)

In [18]:
bookings.shape[0]

3051

In [19]:
# 4.2 Проверим на наличие лишних пробелов, странных символов, дублей в названиях
bookings.columns

Index(['user_id', 'booked_status_user', 'event_id', 'format', 'type',
       'pass_status', 'booked_status_author', 'diff_start'],
      dtype='object')

In [20]:
# 4.3 Посмотрим какие типы данных у колонок, сколько ненулевых значений, сколько памяти жрёт датафрейм
bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3051 entries, 0 to 3050
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   user_id               3051 non-null   int64 
 1   booked_status_user    3051 non-null   object
 2   event_id              3051 non-null   int64 
 3   format                3051 non-null   object
 4   type                  3051 non-null   object
 5   pass_status           3051 non-null   int64 
 6   booked_status_author  3051 non-null   object
 7   diff_start            3051 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 190.8+ KB


In [21]:
# 4.4 Посмотрим обстановку по числовым данны.
bookings.diff_start.describe()

count    3051.000000
mean        8.942314
std        12.702160
min         1.000000
25%         2.000000
50%         5.000000
75%        11.500000
max       151.000000
Name: diff_start, dtype: float64

### `5. Анализ поведения со статусом "Гость"` <a id="step4"></a>

In [22]:
# 5.1 Осмотр данных
bookings_guest = bookings.copy()
bookings_guest.diff_start.describe()

count    3051.000000
mean        8.942314
std        12.702160
min         1.000000
25%         2.000000
50%         5.000000
75%        11.500000
max       151.000000
Name: diff_start, dtype: float64

In [23]:
# 5.2 Проведем анализ среднего кол-ва дней до бронирования у пользователей со статусом Гость

bookings_guest = bookings_guest.query('booked_status_user == "GUEST"')

bookings_guest_online = bookings_guest.copy()
bookings_guest_online = bookings_guest_online.query('format == "ONLINE"')

bookings_guest_offline = bookings_guest.copy()
bookings_guest_offline = bookings_guest_offline.query('format == "OFFLINE"')

bookings_guest_online_paid = bookings_guest.copy()
bookings_guest_online_paid = bookings_guest_online_paid.query('format == "ONLINE"') \
                                                       .query('type == "PAID"')

bookings_guest_online_paid_pass_1 = bookings_guest.copy()
bookings_guest_online_paid_pass_1 = bookings_guest_online_paid_pass_1.query('format == "ONLINE"') \
                                                                     .query('type == "PAID"') \
                                                                     .query('pass_status == 1') \

bookings_guest_online_paid_pass_0 = bookings_guest.copy()
bookings_guest_online_paid_pass_0 = bookings_guest_online_paid_pass_0.query('format == "ONLINE"') \
                                                                     .query('type == "PAID"') \
                                                                     .query('pass_status == 0') \

bookings_guest_online_free = bookings_guest.copy()
bookings_guest_online_free = bookings_guest_online_free.query('format == "ONLINE"') \
                                                       .query('type == "FREE"') \

bookings_guest_offline_paid = bookings_guest.copy()
bookings_guest_offline_paid = bookings_guest_offline_paid.query('format == "OFFLINE"') \
                                                         .query('type == "PAID"') \

bookings_guest_offline_paid_pass_1 = bookings_guest.copy()
bookings_guest_offline_paid_pass_1 = bookings_guest_offline_paid_pass_1.query('format == "OFFLINE"') \
                                                                       .query('type == "PAID"') \
                                                                       .query('pass_status == 1') \

bookings_guest_offline_paid_pass_0 = bookings_guest.copy()
bookings_guest_offline_paid_pass_0 = bookings_guest_offline_paid_pass_0.query('format == "OFFLINE"') \
                                                                       .query('type == "PAID"') \
                                                                       .query('pass_status == 0') \

bookings_guest_offline_free = bookings_guest.copy()
bookings_guest_offline_free = bookings_guest_offline_free.query('format == "OFFLINE"') \
                                                         .query('type == "FREE"')

print(f"Медианное кол-во дней за которые происходит бронирование у Гостей: {bookings_guest.diff_start.median()}")
print()
print("Онлайн")
print(f"Медианное кол-во дней за которые происходит бронирование у Гостей на онлайн: \
{bookings_guest_online.diff_start.median()}")
print(f"Медианное кол-во дней за которые происходит бронирование у Гостей на онлайн (бесплатно): \
{bookings_guest_online_free.diff_start.median()}")
print(f"Медианное кол-во дней за которые происходит бронирование у Гостей на онлайн (платно общее): \
{bookings_guest_online_paid.diff_start.median()}")
print(f"Медианное кол-во дней за которые происходит бронирование у Гостей на онлайн (платно с картой): \
{bookings_guest_online_paid_pass_1.diff_start.median()}")
print(f"Медианное кол-во дней за которые происходит бронирование у Гостей на онлайн (платно без карты): \
{bookings_guest_online_paid_pass_0.diff_start.median()}")

print()
print("Офлайн")
print(f"Медианное кол-во дней за которые происходит бронирование у Гостей на офлайн: \
{bookings_guest_offline.diff_start.median()}")
print(f"Медианное кол-во дней за которые происходит бронирование у Гостей на офлайн (бесплатно): \
{bookings_guest_offline_free.diff_start.median()}")
print(f"Медианное кол-во дней за которые происходит бронирование у Гостей на офлайн (платно общее): \
{bookings_guest_offline_paid.diff_start.median()}")
print(f"Медианное кол-во дней за которые происходит бронирование у Гостей на офлайн (платно с картой): \
{bookings_guest_offline_paid_pass_1.diff_start.median()}")
print(f"Медианное кол-во дней за которые происходит бронирование у Гостей на офлайн (платно без карты): \
{bookings_guest_offline_paid_pass_0.diff_start.median()}")

Медианное кол-во дней за которые происходит бронирование у Гостей: 3.0

Онлайн
Медианное кол-во дней за которые происходит бронирование у Гостей на онлайн: 3.0
Медианное кол-во дней за которые происходит бронирование у Гостей на онлайн (бесплатно): 2.0
Медианное кол-во дней за которые происходит бронирование у Гостей на онлайн (платно общее): 6.0
Медианное кол-во дней за которые происходит бронирование у Гостей на онлайн (платно с картой): 9.5
Медианное кол-во дней за которые происходит бронирование у Гостей на онлайн (платно без карты): 2.0

Офлайн
Медианное кол-во дней за которые происходит бронирование у Гостей на офлайн: 4.0
Медианное кол-во дней за которые происходит бронирование у Гостей на офлайн (бесплатно): 3.0
Медианное кол-во дней за которые происходит бронирование у Гостей на офлайн (платно общее): 7.0
Медианное кол-во дней за которые происходит бронирование у Гостей на офлайн (платно с картой): 20.0
Медианное кол-во дней за которые происходит бронирование у Гостей на офлай

### `6. Анализ поведения со статусом "Амбассадор"` <a id="step5"></a>

In [24]:
# 6.1 Осмотр данных
bookings_ambassador = bookings.copy()
bookings_ambassador.diff_start.describe()

count    3051.000000
mean        8.942314
std        12.702160
min         1.000000
25%         2.000000
50%         5.000000
75%        11.500000
max       151.000000
Name: diff_start, dtype: float64

In [25]:
# 6.2 Проведем анализ среднего кол-ва дней до бронирования у пользователей со статусом Амбассадор

bookings_ambassador = bookings_ambassador.query('booked_status_user == "AMBASSADOR"')

bookings_ambassador_online = bookings_ambassador.copy()
bookings_ambassador_online = bookings_ambassador_online.query('format == "ONLINE"')

bookings_ambassador_offline = bookings_ambassador.copy()
bookings_ambassador_offline = bookings_ambassador_offline.query('format == "OFFLINE"')

bookings_ambassador_online_paid = bookings_ambassador.copy()
bookings_ambassador_online_paid = bookings_ambassador_online_paid.query('format == "ONLINE"') \
                                                                 .query('type == "PAID"') \

bookings_ambassador_online_paid_pass_1 = bookings_ambassador.copy()
bookings_ambassador_online_paid_pass_1 = bookings_ambassador_online_paid_pass_1.query('format == "ONLINE"') \
                                                                               .query('type == "PAID"') \
                                                                               .query('pass_status == 1') \

bookings_ambassador_online_paid_pass_0 = bookings_ambassador.copy()
bookings_ambassador_online_paid_pass_0 = bookings_ambassador_online_paid_pass_0.query('format == "ONLINE"') \
                                                                               .query('type == "PAID"') \
                                                                               .query('pass_status == 0') \

bookings_ambassador_online_free = bookings_ambassador.copy()
bookings_ambassador_online_free = bookings_ambassador_online_free.query('format == "ONLINE"') \
                                                                 .query('type == "FREE"') \

bookings_ambassador_offline_paid = bookings_ambassador.copy()
bookings_ambassador_offline_paid = bookings_ambassador_offline_paid.query('format == "OFFLINE"') \
                                                                   .query('type == "PAID"') \

bookings_ambassador_offline_paid_pass_1 = bookings_ambassador.copy()
bookings_ambassador_offline_paid_pass_1 = bookings_ambassador_offline_paid_pass_1.query('format == "OFFLINE"') \
                                                                                 .query('type == "PAID"') \
                                                                                 .query('pass_status == 1') \

bookings_ambassador_offline_paid_pass_0 = bookings_ambassador.copy()
bookings_ambassador_offline_paid_pass_0 = bookings_ambassador_offline_paid_pass_0.query('format == "OFFLINE"') \
                                                                                 .query('type == "PAID"') \
                                                                                 .query('pass_status == 0') \

bookings_ambassador_offline_free = bookings_ambassador.copy()
bookings_ambassador_offline_free = bookings_ambassador_offline_free.query('format == "OFFLINE"') \
                                                                   .query('type == "FREE"') \

print(f"Медианное  кол-во дней за которые происходит бронирование у Амб.: {bookings_ambassador.diff_start.median()}")
print()
print("Онлайн")
print(f"Медианное  кол-во дней за которые происходит бронирование у Амб. на онлайн: \
{bookings_ambassador_online.diff_start.median()}") 
print(f"Медианное  кол-во дней за которые происходит бронирование у Амб. на онлайн (бесплатно): \
{bookings_ambassador_online_free.diff_start.median()}") 
print(f"Медианное  кол-во дней за которые происходит бронирование у Амб. на онлайн (платно): \
{bookings_ambassador_online_paid.diff_start.median()}")
print(f"Медианное кол-во дней за которые происходит бронирование у Амб. на онлайн (платно с картой): \
{bookings_ambassador_online_paid_pass_1.diff_start.median()}") 
print(f"Медианное кол-во дней за которые происходит бронирование у Амб. на онлайн (платно без карты): \
{bookings_ambassador_online_paid_pass_0.diff_start.median()}")
print()
print("Офлайн")
print(f"Медианное  кол-во дней за которые происходит бронирование у Амб. на офлайн: \
{bookings_ambassador_offline.diff_start.median()}") 
print(f"Медианное  кол-во дней за которые происходит бронирование у Амб. на офлайн (бесплатно): \
{bookings_ambassador_offline_free.diff_start.median()}") 
print(f"Медианное  кол-во дней за которые происходит бронирование у Амб. на офлайн (платно): \
{bookings_ambassador_offline_paid.diff_start.median()}") 
print(f"Медианное кол-во дней за которые происходит бронирование у Амб. на онлайн (платно с картой): \
{bookings_ambassador_online_paid_pass_1.diff_start.median()}") 
print(f"Медианное кол-во дней за которые происходит бронирование у Амб. на онлайн (платно без карты): \
{bookings_ambassador_online_paid_pass_0.diff_start.median()}") 


Медианное  кол-во дней за которые происходит бронирование у Амб.: 6.0

Онлайн
Медианное  кол-во дней за которые происходит бронирование у Амб. на онлайн: 6.0
Медианное  кол-во дней за которые происходит бронирование у Амб. на онлайн (бесплатно): 3.0
Медианное  кол-во дней за которые происходит бронирование у Амб. на онлайн (платно): 9.0
Медианное кол-во дней за которые происходит бронирование у Амб. на онлайн (платно с картой): 9.0
Медианное кол-во дней за которые происходит бронирование у Амб. на онлайн (платно без карты): 6.0

Офлайн
Медианное  кол-во дней за которые происходит бронирование у Амб. на офлайн: 5.0
Медианное  кол-во дней за которые происходит бронирование у Амб. на офлайн (бесплатно): 3.0
Медианное  кол-во дней за которые происходит бронирование у Амб. на офлайн (платно): 6.0
Медианное кол-во дней за которые происходит бронирование у Амб. на онлайн (платно с картой): 9.0
Медианное кол-во дней за которые происходит бронирование у Амб. на онлайн (платно без карты): 6.0


### `7. Ключевые выводы` <a id="step6"></a>

**7.1 Пользователи со статусом "Гость**

**7.1.1 Общее**

- В целом можно сказать, бронируют чаще всего за 3 дня, если кратко.

- 25% Гостей склонны бронировать в последний момент — в день мероприятия или за день до него.

- Еще 25% бронируют за 2–3 дня.

- Еще 25% — за 4–7 дней.

- Оставшиеся 25% бронируют за 8 дней и более.

**7.1.2 По форматам**
- Онлайн — бесплатно	     2 дня	  Увидел, забронировал
- Онлайн — платно без карты  2 дня	  Могу принять участие в любой момент, ближе ко времени заплачу
- Онлайн — платно с картой   10 дней  Вероятно понимание ограниченности предложения

- Офлайн — бесплатно	     3 дня	  Схоже с онлайн-форматом
- Офлайн — платно без карты  6 дней	  Интерес, возможный фактор — ограниченность площадки
- Офлайн — платно с картой	 20 дней   Вероятно понимание ограниченности предложения
 

**7.2 Пользователи со статусом "Амбассадор**

**7.2.1 Общее**

- В целом, чаще всего Амбассадорв бронируют за 6 дней
  
- 25% Амбассадоров бронируют в последний момент — за 2 дня и меньше.

- Еще25% — бронируют за 3–6 дней.

- Еще 25% — за 7–13 дней.

- Еще 25% — за 14 и более дней.

**7.2.2 По форматам**
- Онлайн — бесплатно	                 3 дня	Преимущественно спонтанно, увидел, забронировал
- Онлайн — платно без карты       6 дней	Среднее совсем
- Онлайн — платно с картой         9 дней	Раннее бронирование, вероятно понимание ограниченности
- Офлайн — бесплатно	                 3 дня	Преимущественно спонтанно
- Офлайн — платно без карты      6 дней	Среднее совсем
- Офлайн — платно с картой         9 дней	Раннее бронирование

### `8. Рекомендации` <a id="step7"></a>

- 8.1  В PR-активностях о мероприятии учитывать формат и выстраивать коммуникацию так, чтобы самая "яркая" и ключевая точка промо-активностей приходилась на рубеж бронирования (указаны в разделе "Ключевые моменты" > "По форматам")

- 8.2  Участие в бесплатных мероприятиях люди преимущественно принимают решение спонтанно, — зашел, увидел, забронировал, и в них в большей степени следует упирать именно на эмоциональные факторы. В этой ситуации человек вероятно уже понимает, что у него в эти дни свободно, и следующий фильтр — будет "вау" или — нет.

- 8.3  В цепочке прогрева самые основные факторы принятия решения лучше всего раскрывать в период за 3 дня до мероприятия

- 8.4  SPACE Pass — есть основания говорить о том, что люди понимают ограниченность предложения, бронируют раньше и это время можно использовать да более длинную цепочку коммуникации и прогрева, — в гостях это 10 дней для онлайн-форматов, и 20 дней для офлайн. Отпиарить это можно так — SPACE Pass предоставляет организатору большее время на формирование лояльных отношений.


[В начало](#stepSTART)